In [1]:
!pip install google-play-scraper

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Python312\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'


In [2]:
import google_play_scraper

In [3]:
app_id = 'com.tokopedia.tkpd'

In [4]:
from google_play_scraper import Sort
from google_play_scraper.constants.element import ElementSpecs
from google_play_scraper.constants.regex import Regex
from google_play_scraper.constants.request import Formats
from google_play_scraper.utils.request import post

import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time
import json
from time import sleep
from typing import List, Optional, Tuple

In [5]:
MAX_COUNT_EACH_FETCH = 199


class _ContinuationToken:
    __slots__ = (
        "token",
        "lang",
        "country",
        "sort",
        "count",
        "filter_score_with",
        "filter_device_with",
    )

    def __init__(
        self, token, lang, country, sort, count, filter_score_with, filter_device_with
    ):
        self.token = token
        self.lang = lang
        self.country = country
        self.sort = sort
        self.count = count
        self.filter_score_with = filter_score_with
        self.filter_device_with = filter_device_with


def _fetch_review_items(
    url: str,
    app_id: str,
    sort: int,
    count: int,
    filter_score_with: Optional[int],
    filter_device_with: Optional[int],
    pagination_token: Optional[str],
):
    dom = post(
        url,
        Formats.Reviews.build_body(
            app_id,
            sort,
            count,
            "null" if filter_score_with is None else filter_score_with,
            "null" if filter_device_with is None else filter_device_with,
            pagination_token,
        ),
        {"content-type": "application/x-www-form-urlencoded"},
    )
    match = json.loads(Regex.REVIEWS.findall(dom)[0])

    return json.loads(match[0][2])[0], json.loads(match[0][2])[-2][-1]


def reviews(
    app_id: str,
    lang: str = "en",
    country: str = "us",
    sort: Sort = Sort.MOST_RELEVANT,
    count: int = 100,
    filter_score_with: int = None,
    filter_device_with: int = None,
    continuation_token: _ContinuationToken = None,
) -> Tuple[List[dict], _ContinuationToken]:
    sort = sort.value

    if continuation_token is not None:
        token = continuation_token.token

        if token is None:
            return (
                [],
                continuation_token,
            )

        lang = continuation_token.lang
        country = continuation_token.country
        sort = continuation_token.sort
        count = continuation_token.count
        filter_score_with = continuation_token.filter_score_with
        filter_device_with = continuation_token.filter_device_with
    else:
        token = None

    url = Formats.Reviews.build(lang=lang, country=country)

    _fetch_count = count

    result = []

    while True:
        if _fetch_count == 0:
            break

        if _fetch_count > MAX_COUNT_EACH_FETCH:
            _fetch_count = MAX_COUNT_EACH_FETCH

        try:
            review_items, token = _fetch_review_items(
                url,
                app_id,
                sort,
                _fetch_count,
                filter_score_with,
                filter_device_with,
                token,
            )
        except (TypeError, IndexError):
            #funnan MOD start
            token = continuation_token.token
            continue
            #MOD end

        for review in review_items:
            result.append(
                {
                    k: spec.extract_content(review)
                    for k, spec in ElementSpecs.Review.items()
                }
            )

        _fetch_count = count - len(result)

        if isinstance(token, list):
            token = None
            break

    return (
        result,
        _ContinuationToken(
            token, lang, country, sort, count, filter_score_with, filter_device_with
        ),
    )


def reviews_all(app_id: str, sleep_milliseconds: int = 0, **kwargs) -> list:
    kwargs.pop("count", None)
    kwargs.pop("continuation_token", None)

    continuation_token = None

    result = []

    while True:
        _result, continuation_token = reviews(
            app_id,
            count=MAX_COUNT_EACH_FETCH,
            continuation_token=continuation_token,
            **kwargs
        )

        result += _result

        if continuation_token.token is None:
            break

        if sleep_milliseconds:
            sleep(sleep_milliseconds / 1000)

    return result

In [6]:
reviews_count = 10000

In [7]:
result = []
continuation_token = None


with tqdm(total=reviews_count, position=0, leave=True) as pbar:
    while len(result) < reviews_count:
        new_result, continuation_token = reviews(
            app_id,
            continuation_token=continuation_token,
            lang='id', #The language of review
            country='id', #Country for which you want to scrape
            sort=Sort.NEWEST,
            filter_score_with=None,
            count=199 #No need to change this
        )
        if not new_result:
            break
        result.extend(new_result)
        pbar.update(len(new_result))

  0%|          | 0/10000 [00:00<?, ?it/s]

10149it [00:44, 230.38it/s]                          


In [8]:
import numpy as np
df = pd.DataFrame(np.array(result),columns=['review'])

df = df.join(pd.DataFrame(df.pop('review').tolist()))

df.head(20)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,a09c9851-8816-490e-8ac6-1c0c8560fa9e,Evan sur Van,https://play-lh.googleusercontent.com/a/ACg8oc...,Jelek nggak bisa cod,1,0,None,2024-10-18 22:43:35,"Halo Toppers, terima kasih atas rating yang ka...",2024-10-18 22:54:01,None
1,2f08e68e-0ea1-4291-b90b-af4e9621aa88,fadhilah setyoarum,https://play-lh.googleusercontent.com/a-/ALV-U...,"MASIH ADA KUPON TAPI MALAH GA BISA DIPAKAII, S...",1,0,None,2024-10-18 22:43:00,"Halo Toppers, terima kasih atas rating yang ka...",2024-10-18 22:54:03,None
2,5803ebce-24a7-4589-b4e1-fbf104f2eaef,Maha Ansori,https://play-lh.googleusercontent.com/a/ACg8oc...,mantap,5,0,3.285.0,2024-10-18 22:37:03,Terima kasih sudah mempercayakan Tokopedia seb...,2024-10-18 22:54:08,3.285.0
3,3a93c876-d5bb-4db8-bdb7-709f30438892,wilsya syabilla,https://play-lh.googleusercontent.com/a-/ALV-U...,manjut,5,0,3.285.0,2024-10-18 22:34:42,Terima kasih sudah mempercayakan Tokopedia seb...,2024-10-18 22:54:09,3.285.0
4,f1612529-a3b5-46b0-be6e-4af2e7a67afe,Fatih Nabiel,https://play-lh.googleusercontent.com/a-/ALV-U...,"Woy, masa gak ada fitur delete pada barang rek...",1,0,3.285.0,2024-10-18 22:32:56,"Halo Toppers, terima kasih atas rating yang ka...",2024-10-18 22:40:06,3.285.0
5,dc7c763f-e4d0-4fa7-b489-d12322c12817,Duwi Susilo rhmad,https://play-lh.googleusercontent.com/a/ACg8oc...,Payah sumpah toko pedia payah..,1,0,3.283.0,2024-10-18 22:32:02,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-10-18 22:40:07,3.283.0
6,e957a619-475a-44b0-adc9-364eedc8b81f,Mustofa Jubaedah,https://play-lh.googleusercontent.com/a-/ALV-U...,Alhamdulillah byk promo'a,5,0,3.139,2024-10-18 22:30:33,"Toppers, terima kasih atas rating yang kamu be...",2024-10-18 22:34:01,3.139
7,b10460b6-1e11-4521-950f-7a5fb44c2182,Arsenio,https://play-lh.googleusercontent.com/a-/ALV-U...,sangat menyenangkan,5,0,3.283.0,2024-10-18 22:29:56,Terima kasih sudah mempercayakan Tokopedia seb...,2024-10-18 22:34:03,3.283.0
8,8f7f05d6-6db3-4662-b53e-9060b760440c,Muhammad Karuniamuda,https://play-lh.googleusercontent.com/a/ACg8oc...,mantapp bangettt apk nyaaa😆😆😆😆😆,5,0,3.284.0,2024-10-18 22:29:16,Terima kasih sudah mempercayakan Tokopedia seb...,2024-10-18 22:34:05,3.284.0
9,9d21e10a-6481-4b99-9ada-8f43f8d0075b,D 005,https://play-lh.googleusercontent.com/a-/ALV-U...,Terlalu sering update bikin berat hp,2,0,3.273.2,2024-10-18 22:29:04,"Halo Toppers, terima kasih atas rating yang ka...",2024-10-18 22:34:06,3.273.2


In [9]:
df.to_csv("01_hasil-scrap-tokopedia-murni.csv", index = False) #simpan csv

In [10]:
len(df)

10149

In [11]:
df[['userName', 'score','at', 'content']].head(20)  #dari scrapping tsb didapatkan banyak sekali kolom, kemudian kolom" tsb kita filter
                                                        #sehingga didapatkan kolom username, score, at dan content

,userName,score,at,content
0,Evan sur Van,1,2024-10-18 22:43:35,Jelek nggak bisa cod
1,fadhilah setyoarum,1,2024-10-18 22:43:00,"MASIH ADA KUPON TAPI MALAH GA BISA DIPAKAII, S..."
2,Maha Ansori,5,2024-10-18 22:37:03,mantap
3,wilsya syabilla,5,2024-10-18 22:34:42,manjut
4,Fatih Nabiel,1,2024-10-18 22:32:56,"Woy, masa gak ada fitur delete pada barang rek..."
5,Duwi Susilo rhmad,1,2024-10-18 22:32:02,Payah sumpah toko pedia payah..
6,Mustofa Jubaedah,5,2024-10-18 22:30:33,Alhamdulillah byk promo'a
7,Arsenio,5,2024-10-18 22:29:56,sangat menyenangkan
8,Muhammad Karuniamuda,5,2024-10-18 22:29:16,mantapp bangettt apk nyaaa😆😆😆😆😆
9,D 005,2,2024-10-18 22:29:04,Terlalu sering update bikin berat hp


In [12]:
df=df[['content', 'score']]#karena kita hanya membutuhkan kolom content dan score maka kita lakukan filter kolom lgi hing
df.head(20)

,content,score
0,Jelek nggak bisa cod,1
1,"MASIH ADA KUPON TAPI MALAH GA BISA DIPAKAII, S...",1
2,mantap,5
3,manjut,5
4,"Woy, masa gak ada fitur delete pada barang rek...",1
5,Payah sumpah toko pedia payah..,1
6,Alhamdulillah byk promo'a,5
7,sangat menyenangkan,5
8,mantapp bangettt apk nyaaa😆😆😆😆😆,5
9,Terlalu sering update bikin berat hp,2


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10149 entries, 0 to 10148
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  10149 non-null  object
 1   score    10149 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 158.7+ KB


In [14]:
df.isna()

,content,score
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
10144,False,False
10145,False,False
10146,False,False
10147,False,False


In [15]:
df.describe()

,score
count,10149.000000
mean,3.361021
std,1.846793
min,1.000000
25%,1.000000
50%,5.000000
75%,5.000000
max,5.000000


In [16]:
df.isnull().sum()

content    0
score      0
dtype: int64

In [17]:
df.to_csv("02_hasil-scrap-tokopedia.csv", index = False) #simpan csv

# **PREPROCESSING**

Case Folding

In [18]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    return df

In [19]:
df['text_clean'] = df['content'].str.lower()
df['text_clean']
data_clean = clean_text(df, 'content', 'text_clean')
data_clean.head(20)

,content,score,text_clean
0,Jelek nggak bisa cod,1,jelek nggak bisa cod
1,"MASIH ADA KUPON TAPI MALAH GA BISA DIPAKAII, S...",1,masih ada kupon tapi malah ga bisa dipakaii se...
2,mantap,5,mantap
3,manjut,5,manjut
4,"Woy, masa gak ada fitur delete pada barang rek...",1,woy masa gak ada fitur delete pada barang reko...
5,Payah sumpah toko pedia payah..,1,payah sumpah toko pedia payah
6,Alhamdulillah byk promo'a,5,alhamdulillah byk promoa
7,sangat menyenangkan,5,sangat menyenangkan
8,mantapp bangettt apk nyaaa😆😆😆😆😆,5,mantapp bangettt apk nyaaa
9,Terlalu sering update bikin berat hp,2,terlalu sering update bikin berat hp


Stopword Removal

In [20]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('indonesian')
data_clean['text_stopword'] = data_clean['text_clean'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
data_clean.head(50)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,content,score,text_clean,text_stopword
0,Jelek nggak bisa cod,1,jelek nggak bisa cod,jelek nggak cod
1,"MASIH ADA KUPON TAPI MALAH GA BISA DIPAKAII, S...",1,masih ada kupon tapi malah ga bisa dipakaii se...,kupon ga dipakaii dipakai baranganya ngga nyas...
2,mantap,5,mantap,mantap
3,manjut,5,manjut,manjut
4,"Woy, masa gak ada fitur delete pada barang rek...",1,woy masa gak ada fitur delete pada barang reko...,woy gak fitur delete barang rekomendasi sih ud...
5,Payah sumpah toko pedia payah..,1,payah sumpah toko pedia payah,payah sumpah toko pedia payah
6,Alhamdulillah byk promo'a,5,alhamdulillah byk promoa,alhamdulillah byk promoa
7,sangat menyenangkan,5,sangat menyenangkan,menyenangkan
8,mantapp bangettt apk nyaaa😆😆😆😆😆,5,mantapp bangettt apk nyaaa,mantapp bangettt apk nyaaa
9,Terlalu sering update bikin berat hp,2,terlalu sering update bikin berat hp,update bikin berat hp


In [21]:
data_clean.to_csv("03_hasil-stopword-filter.csv", index = False) #simpan csv
#process this result with 01_1_slangword_stopword_removal.ipynb

**Tokenizing**


In [51]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
data_clean['text_tokens'] = data_clean['text_stopword'].apply(lambda x: word_tokenize(x))
data_clean.head(20)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,content,score,text_clean,text_StopWord,text_stopword,text_tokens
0,good 👍,5,good,good,good,[good]
1,KEMBALIKAN UI LAMA!! UI BARU (18 OKTOBER 2024)...,1,kembalikan ui lama ui baru oktober sangat je...,kembalikan ui ui oktober jelek pemberitahuan d...,kembalikan ui ui oktober jelek pemberitahuan d...,"[kembalikan, ui, ui, oktober, jelek, pemberita..."
2,bagus,5,bagus,bagus,bagus,[bagus]
3,Sudah ber kali kali saya gagal up date Apakah ...,2,sudah ber kali kali saya gagal up date apakah ...,ber kali kali gagal up date solusi,ber kali kali gagal up date solusi,"[ber, kali, kali, gagal, up, date, solusi]"
4,Lama prosesnya dr tokopedia Kurir bermasalah p...,1,lama prosesnya dr tokopedia kurir bermasalah p...,prosesnya dr tokopedia kurir bermasalah aja ba...,prosesnya dr tokopedia kurir bermasalah aja ba...,"[prosesnya, dr, tokopedia, kurir, bermasalah, ..."
5,Tokopedia pliiss jangan kerjasama lagi dengan ...,3,tokopedia pliiss jangan kerjasama lagi dengan ...,tokopedia pliiss kerjasama id express lamaaa b...,tokopedia pliiss kerjasama id express lamaaa b...,"[tokopedia, pliiss, kerjasama, id, express, la..."
6,Ok,5,ok,ok,ok,[ok]
7,Aplikasi bodat ini temen gua di PHP in promo g...,1,aplikasi bodat ini temen gua di php in promo g...,aplikasi bodat temen gua php in promo gabisa u...,aplikasi bodat temen gua php in promo gabisa u...,"[aplikasi, bodat, temen, gua, php, in, promo, ..."
8,aplikasi nya udh bagus tpi yg jadi masalah dis...,3,aplikasi nya udh bagus tpi yg jadi masalah dis...,aplikasi nya udh bagus tpi yg diskon nya vepat...,aplikasi nya udh bagus tpi yg diskon nya vepat...,"[aplikasi, nya, udh, bagus, tpi, yg, diskon, n..."
9,"Gw dapet voucher 20rb, tiba2 dibatalin sistem,...",1,gw dapet voucher rb tiba dibatalin sistem pemb...,gw dapet voucher rb dibatalin sistem pembayara...,gw dapet voucher rb dibatalin sistem pembayara...,"[gw, dapet, voucher, rb, dibatalin, sistem, pe..."


In [52]:
data_clean.to_csv("04_hasil-tokenisasi.csv", index = False) #simpan csv

**Stemming**

In [48]:
!pip install Sastrawi

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Python312\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'


In [49]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [53]:
#-----------------STEMMING -----------------
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
#import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}
hitung=0

for document in data_clean['text_tokens']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    hitung+=1
    print(hitung,":",term,":" ,term_dict[term])

print(term_dict)
print("------------------------")

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]


#script ini bisa dipisah dari eksekusinya setelah pembacaaan term selesai
data_clean['text_steamindo'] = data_clean['text_tokens'].apply(lambda x:' '.join(get_stemmed_term(x)))
data_clean.head(20)

9669
------------------------
1 : good : good
2 : kembalikan : kembali
3 : ui : ui
4 : oktober : oktober
5 : jelek : jelek
6 : pemberitahuan : pemberitahuan
7 : digabung : gabung
8 : pesan : pesan
9 : membedakan : beda
10 : bagus : bagus
11 : ber : ber
12 : kali : kali
13 : gagal : gagal
14 : up : up
15 : date : date
16 : solusi : solusi
17 : prosesnya : proses
18 : dr : dr
19 : tokopedia : tokopedia
20 : kurir : kurir
21 : bermasalah : masalah
22 : aja : aja
23 : barang : barang
24 : gak : gak
25 : tujuan : tuju
26 : info : info
27 : nya : nya
28 : diterima : terima
29 : bersangkutan : sangkut
30 : kagak : kagak
31 : tindakan : tindak
32 : lambat : lambat
33 : pakai : pakai
34 : pliiss : pliiss
35 : kerjasama : kerjasama
36 : id : id
37 : express : express
38 : lamaaa : lamaaa
39 : banget : banget
40 : kurirnya : kurir
41 : pesen : sen
42 : tnggal : tnggal
43 : oct : oct
44 : smpek : smpek
45 : tanggal : tanggal
46 : ok : ok
47 : aplikasi : aplikasi
48 : bodat : bodat
49 : temen : tem

,content,score,text_clean,text_StopWord,text_stopword,text_tokens,text_steamindo
0,good 👍,5,good,good,good,[good],good
1,KEMBALIKAN UI LAMA!! UI BARU (18 OKTOBER 2024)...,1,kembalikan ui lama ui baru oktober sangat je...,kembalikan ui ui oktober jelek pemberitahuan d...,kembalikan ui ui oktober jelek pemberitahuan d...,"[kembalikan, ui, ui, oktober, jelek, pemberita...",kembali ui ui oktober jelek pemberitahuan gabu...
2,bagus,5,bagus,bagus,bagus,[bagus],bagus
3,Sudah ber kali kali saya gagal up date Apakah ...,2,sudah ber kali kali saya gagal up date apakah ...,ber kali kali gagal up date solusi,ber kali kali gagal up date solusi,"[ber, kali, kali, gagal, up, date, solusi]",ber kali kali gagal up date solusi
4,Lama prosesnya dr tokopedia Kurir bermasalah p...,1,lama prosesnya dr tokopedia kurir bermasalah p...,prosesnya dr tokopedia kurir bermasalah aja ba...,prosesnya dr tokopedia kurir bermasalah aja ba...,"[prosesnya, dr, tokopedia, kurir, bermasalah, ...",proses dr tokopedia kurir masalah aja barang g...
5,Tokopedia pliiss jangan kerjasama lagi dengan ...,3,tokopedia pliiss jangan kerjasama lagi dengan ...,tokopedia pliiss kerjasama id express lamaaa b...,tokopedia pliiss kerjasama id express lamaaa b...,"[tokopedia, pliiss, kerjasama, id, express, la...",tokopedia pliiss kerjasama id express lamaaa b...
6,Ok,5,ok,ok,ok,[ok],ok
7,Aplikasi bodat ini temen gua di PHP in promo g...,1,aplikasi bodat ini temen gua di php in promo g...,aplikasi bodat temen gua php in promo gabisa u...,aplikasi bodat temen gua php in promo gabisa u...,"[aplikasi, bodat, temen, gua, php, in, promo, ...",aplikasi bodat temen gua php in promo gabisa u...
8,aplikasi nya udh bagus tpi yg jadi masalah dis...,3,aplikasi nya udh bagus tpi yg jadi masalah dis...,aplikasi nya udh bagus tpi yg diskon nya vepat...,aplikasi nya udh bagus tpi yg diskon nya vepat...,"[aplikasi, nya, udh, bagus, tpi, yg, diskon, n...",aplikasi nya udh bagus tpi yg diskon nya vepat...
9,"Gw dapet voucher 20rb, tiba2 dibatalin sistem,...",1,gw dapet voucher rb tiba dibatalin sistem pemb...,gw dapet voucher rb dibatalin sistem pembayara...,gw dapet voucher rb dibatalin sistem pembayara...,"[gw, dapet, voucher, rb, dibatalin, sistem, pe...",gw dapet voucher rb dibatalin sistem bayar via...


In [54]:
data_clean.to_csv('05_hasil_stemming_tokopedia.csv', index= False) #kemudian simpan hasil text preprocessing ke file csv